# Transformers 라이브러리 Trainer 클래스
- Trainer클래스는 PyTorch에서 기능 전체 학습을 위한 API를 제공
- 사용자는 직접 훈련 루프를 작성할 필요 없이 더욱 간편하게 학습을 시킬 수 있음
- Trainer는 TrainingArguments 클래스와 함께 사용되며, 이 클래스는 모델 학습 방법을 사용자 정의하는 광범위한 옵션을 제공
- Trainer클래스는 Transformers 라이브러리의 모델에 최적화

In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "../data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
df = pd.read_csv(f"{DATA_PATH}imdb_small.csv")
df.shape

(748, 2)

In [4]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [5]:
df["sentiment"].mean()

0.516042780748663

# 허깅페이스의 datasets 라이브러리
- 자연어 뿐만 아니라, 오디오, 이미지 등 다양한 데이터셋을 쉽게 공유하고 접근할 수 있게 해주는 라이브러리
- 한 줄의 코드만으로 데이터셋을 로드하거나 학습 가능한 데이터셋 생성
- 설치
```
pip install datasets
```

## load_dataset
- 허깅페이스의 허브에서 데이터셋을 로드하거나, 로컬에서 데이터셋을 로드
- 주요 파라미터
    - path: 로컬 데이터셋 유형("csv", "json", "text" 등)
        - 허깅페이스에서 데이터셋 로드 시 데이터셋 ID 명을 지정
    - data_files: 로컬에서 데이터 로드 시 경로 지정

In [6]:
from datasets import load_dataset
dt = load_dataset("csv", data_files=f"{DATA_PATH}imdb_small.csv", split="train") # load_dataset("HuggingFaceFW/fineweb-2")
dt

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 748
})

## Dataset 클래스
- 단일 세트 객체

In [7]:
dt.num_rows

748

- 인덱싱, 슬라이싱, 멀티 인덱싱 가능

In [8]:
dt[0]

{'review': 'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
 'sentiment': 0}

In [9]:
dt[:2]

{'review': ['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
  'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  '],
 'sentiment': [0, 0]}

In [10]:
dt[[0, 1, 2]]

{'review': ['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
  'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  ',
  'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  '],
 'sentiment': [0, 0, 0]}

## DatasetDict 클래스
- 여러 데이터셋 객체를 하나의 객체로 관리

In [11]:
data_dict = {
    "train": f"{DATA_PATH}imdb_small.csv",
    "test": f"{DATA_PATH}imdb_small.csv"
}
dt = load_dataset("csv", data_files=data_dict)
dt

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 748
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 748
    })
})

In [12]:
dt["train"]

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 748
})

In [13]:
dt = load_dataset("imdb")
dt

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

## DataFrame to Dataset

In [14]:
from datasets import Dataset

In [15]:
dt = Dataset.from_pandas(df)
dt

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 748
})

In [16]:
dt[0]

{'review': 'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
 'sentiment': 0}

## Dataset 클래스 객체의 map 메서드
- 데이터셋 객체의 데이터를 전처리하는데 사용
- 전처리하는 콜백함수 전달

In [17]:
from transformers import AutoTokenizer

In [18]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [19]:
dt = dt.map(lambda x: tokenizer(x["review"], truncation=True, padding=True), batched=True)

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

In [20]:
dt[0].keys()

dict_keys(['review', 'sentiment', 'input_ids', 'token_type_ids', 'attention_mask'])

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# TrainingArguments 클래스
- Trainer가 학습 및 평가에 사용하는 하이퍼파라미터를 정의하는 클래스


## 주요 파라미터
- output_dir
    - 첫번째 파라미터로 학습 과정에서 모델 관련 설정과 가중치 등을 저장할 경로 지정
- per_device_train_batch_size
    - 학습데이터 배치 사이즈(기본값: 8)
- per_device_eval_batch_size
    - 평가데이터 배치 사이즈(기본값: 8)
- learning_rate
    - `AdamW`의 학습률(기본값: `5e-5`)
- weight_decay
    - 가중치 감소(L2 정규화)를 적용하는데 사용되는 가중치 감소 계수(기본값: 0)
- num_train_epochs
    - 훈련 에폭수(기본값: 3)
- lr_scheduler_type
    - learning rate scheduler 지정(기본값: "linear")
    - "linear": 학습률이 선형적으로 감소
    - "cosine": 코사인 함수를 기반으로 학습률이 감소
    - "cosine_with_restarts": 코사인 함수 기반으로 학습률이 감소하며, 주기적으로 재시작
    - "polynomial": 다항식을 기반으로 학습률이 감소
    - "constant": 학습률 유지
    - "constant_with_warmup": 학습률이 0부터 조금씩 증가하다가 학습률 유지
    - "inverse_sqrt": 학습률이 역제곱 함수에 따라 감소
    - "reduce_lr_on_plateau": 모델 성능 개선이 일정 기간 동안 정체되었을 때 학습률을 감소
    - "cosine_with_min_lr": 코사인 함수 형태로 점진적으로 감소시키되, 학습률이 특정 최소값 이하로 떨어지지 않도록 설정
- use_cpu
    - True 전달 시 무조건 CPU 학습(기본값: False)
- seed
    - 시드값(기본값: 42)
- eval_strategy
    - 평가를 언제 수행할 지를 결정
    - "no": 기본값으로 평가 수행 안함.
    - "steps": 스텝마다 수행
        - eval_steps 파라미터도 설정해야 함
    - "epoch": 에폭마다 수행
- logging_strategy
    - 로그를 언제 기록할 지를 결정
    - "no": 로그를 안 남김
    - "steps": 기본값으로 스텝마다 로깅
        - logging_steps 파라미터도 설정해야 함
    - "epoch": 에폭마다 로깅
- save_strategy
    - 학습 중 모델의 상태를 언제 저장할 지를 결정
    - "no": 저장 안함
    - "steps": 기본값으로 스텝마다 저장
        - save_steps 파라미터도 설정해야 함
    - "epoch": 에폭마다 저장
- load_best_model_at_end
    - True 전달 시 가장 성능 좋은 모델 로드(기본값: False)
- metric_for_best_model
    - load_best_model_at_end 파라미터의 기준이 되는 평가지표
    - load_best_model_at_end True 전달 시 기본으로 loss 가 결정 됨
- greater_is_better
    - metric_for_best_model 평가지표가 높을 수록 좋은지 낮을 수록 좋은지에 대한 여부
    - 높을 수록 좋을 경우 True
    - 낮을 수록 좋을 경우 False


In [22]:
from transformers import TrainingArguments

In [23]:
train_args_params = {
    "output_dir": "../output",
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "num_train_epochs": 5,
    "eval_strategy": "epoch",
    "logging_strategy": "epoch",
    "save_strategy": "epoch",
    "load_best_model_at_end": True,
    "metric_for_best_model": "accuracy",
    "greater_is_better": True,
    "report_to": "none"
}

In [24]:
train_args = TrainingArguments(**train_args_params)

# Trainer 클래스
- 모델, TrainingArguments의 객체, 학습 데이터, 검증 데이터 등을 전달하여 학습을 수행하는 클래스

## 주요 파라미터
- model
    - 첫번째 파라미터로 모델 객체 전달
- args
    - 두번째 파라미터로 TrainingArguments의 객체 전달
- train_dataset
    - 학습용 dataset 라이브러리의 Dataset 객체 전달
- eval_dataset
    - 검증용 dataset 라이브러리의 Dataset 객체 전달
- compute_metrics
    - 평가함수 객체 전달
- optimizers
    - 옵티마이저와 스케쥴러 클래스를 튜플에 묶어 전달
    - 기본값: `(None, None)`


In [25]:
from transformers import Trainer

## compute_metrics 파라미터에 전달하는 콜백함수
- 콜백함수 작성 시 평가지표 이름을 key로, 평가값을 value로 해서 dict 형태로 반환
- trainer가 인수로 EvalPrediction 객체를 전달
- EvalPrediction 객체의 주요 속성
    - label_ids: 정답데이터
    - predictions: 모델의 예측값

In [26]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_prediction):
    true = eval_prediction.label_ids
    pred = eval_prediction.predictions
    pred = np.argmax(pred, axis=1)
    score = accuracy_score(true, pred)
    return {"accuracy": score}

In [27]:
from sklearn.model_selection import KFold

cv = KFold(5, shuffle=True, random_state=SEED)

In [28]:
df = df.rename(columns={"sentiment": "label"}) # 정답 컬럼 이름은 label이어야 함
df.head()

,review,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


# 학습하기

In [29]:
is_holdout = True
reset_seeds(SEED)
best_score_list = []
preprocess_data = lambda x: tokenizer(x["review"], truncation=True, padding=True)
for i, (tri, vai) in enumerate(cv.split(df)):
    x_train = df.iloc[tri].reset_index(drop=True)
    x_valid = df.iloc[vai].reset_index(drop=True)
    train_dt = Dataset.from_pandas(x_train).map(preprocess_data, batched=True)
    valid_dt = Dataset.from_pandas(x_valid).map(preprocess_data, batched=True)

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    trainer = Trainer(
        model,
        train_args,
        train_dataset=train_dt,
        eval_dataset=valid_dt,
        compute_metrics=compute_metrics,
    )
    trainer.train()

    if is_holdout:
        break

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/190 [00:00<?, ?it/s]

{'loss': 0.4418, 'grad_norm': 16.916505813598633, 'learning_rate': 4e-05, 'epoch': 1.0}


...

{'eval_loss': 0.2603275179862976, 'eval_accuracy': 0.9533333333333334, 'eval_runtime': 1.0087, 'eval_samples_per_second': 148.713, 'eval_steps_per_second': 9.914, 'epoch': 3.0}
{'loss': 0.0022, 'grad_norm': 0.032854508608579636, 'learning_rate': 1e-05, 'epoch': 4.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.28544193506240845, 'eval_accuracy': 0.9533333333333334, 'eval_runtime': 2.5055, 'eval_samples_per_second': 59.868, 'eval_steps_per_second': 3.991, 'epoch': 4.0}
{'loss': 0.0013, 'grad_norm': 0.02509978599846363, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.2871916890144348, 'eval_accuracy': 0.9533333333333334, 'eval_runtime': 0.7734, 'eval_samples_per_second': 193.957, 'eval_steps_per_second': 12.93, 'epoch': 5.0}
{'train_runtime': 1045.6171, 'train_samples_per_second': 2.86, 'train_steps_per_second': 0.182, 'train_loss': 0.11860822025490435, 'epoch': 5.0}


- 평가 메서드

In [30]:
trainer.evaluate(valid_dt)

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.2603275179862976,
 'eval_accuracy': 0.9533333333333334,
 'eval_runtime': 1.3172,
 'eval_samples_per_second': 113.875,
 'eval_steps_per_second': 7.592,
 'epoch': 5.0}

- 예측하기

In [31]:
pred_output = trainer.predict(valid_dt)

  0%|          | 0/10 [00:00<?, ?it/s]

- 예측값

In [32]:
pred_output.predictions.shape

(150, 2)

- 클래스 결정

In [33]:
pred = np.argmax(pred_output.predictions, axis=1)
pred

array([0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], dtype=int64)

- 1에 대한 확률

In [34]:
softmax = torch.nn.Softmax(dim=1)
pred = softmax(torch.Tensor(pred_output.predictions)).numpy()[:, 1]
pred

array([6.8187359e-04, 9.9880421e-01, 9.9866414e-01, 9.9875116e-01,
       7.8044331e-04, 1.1021863e-03, 7.4317749e-04, 9.9801338e-01,
       9.9881709e-01, 6.8141398e-04, 7.4248842e-04, 1.7722768e-03,
       9.9873978e-01, 2.1967313e-03, 7.7036885e-04, 8.3575217e-04,
       1.6132065e-03, 7.9380936e-04, 7.1763305e-04, 6.9405645e-04,
       6.8697258e-04, 8.6454808e-04, 8.0225983e-04, 9.3578344e-04,
       9.9883443e-01, 8.8868558e-04, 8.9152117e-04, 9.9241829e-01,
       8.3344354e-04, 7.0165761e-04, 7.6152687e-04, 9.9882549e-01,
       9.9854636e-01, 9.9827385e-01, 9.9877614e-01, 9.9876559e-01,
       7.8043109e-04, 8.1870757e-04, 9.9879467e-01, 9.9889982e-01,
       9.9883157e-01, 9.9873763e-01, 1.5556649e-03, 9.9876261e-01,
       9.9887520e-01, 1.8325130e-03, 9.3449553e-04, 6.7368924e-04,
       1.0885239e-03, 6.2111538e-04, 6.8476825e-04, 9.9880135e-01,
       9.9856204e-01, 8.0726133e-04, 9.9672264e-01, 9.9789149e-01,
       7.9825934e-04, 7.5864332e-04, 6.7361892e-04, 9.9880159e

- 체크포인트로 모델 불러오기

In [35]:
trainer.state.best_model_checkpoint

'../output\\checkpoint-114'

In [36]:
best_model = AutoModelForSequenceClassification.from_pretrained(trainer.state.best_model_checkpoint)

In [37]:
train_arg = TrainingArguments(output_dir="tmp", report_to="none")
trainer = Trainer(best_model, train_arg)

In [38]:
trainer.predict(valid_dt)

  0%|          | 0/19 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[ 3.5995028, -3.6904812],
       [-3.157108 ,  3.5706909],
       [-3.1137795,  3.50306  ],
       [-3.1199594,  3.5643122],
       [ 3.5554693, -3.5993984],
       [ 3.4656131, -3.3437417],
       [ 3.6209025, -3.5829303],
       [-2.931353 ,  3.2879996],
       [-3.194396 ,  3.5441604],
       [ 3.6274705, -3.6631875],
       [ 3.6012652, -3.6034954],
       [ 3.213979 , -3.1197362],
       [-3.1175814,  3.5575871],
       [ 3.1098452, -3.0087404],
       [ 3.576908 , -3.5909617],
       [ 3.523872 , -3.562471 ],
       [ 3.327077 , -3.1008396],
       [ 3.5399146, -3.5979593],
       [ 3.613393 , -3.6254416],
       [ 3.6092257, -3.6630363],
       [ 3.6124766, -3.6700532],
       [ 3.544362 , -3.508076 ],
       [ 3.572153 , -3.5551217],
       [ 3.5354335, -3.4377575],
       [-3.1947968,  3.5586004],
       [ 3.5230808, -3.5017967],
       [ 3.5467975, -3.4748921],
       [-2.222183 ,  2.6522303],
       [ 3.5454772, -3.5436335],
       [ 3.604